In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

df = pd.DataFrame(np.random.randint(0, int(1e8), (10000, 1000)))

# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
df.groupby(0).progress_apply(lambda x: x**2)

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
letters = ['A', 'B', 'D', 'E', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'Sade', 'Shin', 'T', 'Tet', 'U', 'W', 'Y', 'Z']
ur3 = []
for l in letters:
    response = requests.get("http://altorient.gko.uni-leipzig.de/SAD/" + l + ".htm")
    doc = response.content
    new_doc = UnicodeDammit.detwingle(doc)
    soup = BeautifulSoup(new_doc, 'html.parser')
    for n in soup.find_all("p"):
        if "Ur III" in n.get_text(): 
            ur3.append(n.get_text())

In [ ]:
ur3

In [ ]:
ur3_2 = []
for i in ur3:
    i = i.replace('â€�', '"')
    i = i.replace('â€œ', '"')
    i = i.replace('tÃºg', '{tug₂}')
    i = i.replace('\n', ' ')
    i = i.replace('Ãº', 'ú')
    ur3_2.append(i)

In [ ]:
ur3_2

In [ ]:
new_doc.decode("utf-8")

In [ ]:
import json

In [ ]:
d = '../2_1_Data_Acquisition_ORACC/jsonzip/obmc/corpusjson'
with open(d + '/X000011.json', encoding = 'utf-8') as j:
    data_json = json.load(j)

In [ ]:
data = data_json['cdl']
type(data)

In [ ]:
for l in data:
    if 'linkbase' in l:
        print(l['linkbase'])

In [ ]:
import zipfile
file = "../2_1_Data_Acquisition_ORACC/jsonzip/epsd2-literary.zip"    
z = zipfile.ZipFile(file)
st = z.read("epsd2/literary/corpusjson/Q000616.json").decode("utf-8") 
text = json.loads(st)
cdl = text['cdl']
for n in cdl:
    if 'linkbase' in n:
        linkbase = n['linkbase']

In [ ]:
linkbase

In [1]:
import re
letters = r'a-zḫĝŋṣšṭA-ZḪĜŊṢŠṬ'
lettersX = re.compile(fr'(?<=[{letters}])X') # capital X preceded by a letter
lettersNo = re.compile(fr'[{letters}](\d+|x)') # any sequence of digits, or X, preceded by a letter

ascind, uniind = '0123456789x', '₀₁₂₃₄₅₆₇₈₉ₓ'
transind = str.maketrans(ascind, uniind) # translation table for index numbers

In [4]:
with open('Q000759.atf',  'r', encoding='utf8') as w:
    l = w.read().splitlines()

In [9]:
new = []
for line in l:
    if line[0] in '0123456789':
        line = re.sub(lettersNo, lambda m: m.group().translate(transind), line)
    new.append(line)

In [10]:
new

['&Q000759 = Edubba R',
 '@composite',
 '#project: epsd2/literary',
 '#atf: use unicode',
 '#atf: use legacy',
 '#atf: use math',
 '$ (Edition after A. Gadotti and A. Kleinerman in JAOS 137, 2017, 89-116)',
 '@div segment A',
 '1. lú tur dumu é-dub-ba-me-en dumu é-dub-ba-me-en',
 '2. tukum-bi dumu é-dub-ba-me-en',
 '3. a-na-àm á-áŋ-ŋá é-dub-ba-a',
 '4. tukum-bi á-áŋ-ŋá# é#-dub-ba èn ma-tar-re',
 '5. ga-ba-e-ŋar {d}utu-è-ta ud é-gar₈-e gi₄-a-šè',
 '6. inim-inim-da nu-mu-da-til-le',
 '7. á-áŋ-ŋá é-dub-ba a-ab-zu an-zag nu-zu-a',
 '8. dím-ma nu-di-da-gin₇ di-di-da nu-ub-zu',
 '9. é-dub-ba-šè ŋen-a-ŋu₁₀-ne',
 '10. gi₆ 1/3-àm a an-da-ŋál gú-ŋu₁₀ ab-zi-zi-in ka-ŋu₁₀ ab-ak-en',
 '11. ka-ŋu₁₀ ub-ta-ak-an',
 '12. gi-dub-ba-ŋu₁₀ im-šu-ŋu₁₀ níŋ-gù-dé-ŋu₁₀',
 '13. ù níŋ-al-di é-dub-ba-a-ke₄ šu ba-ti',
 '14. é-dub-ba-šè sun₅-na-bi al-du-un igi nu-bar!-bar!-re-en',
 '15. é-dub-ba-a-šè ku₄-ku₄-da-ŋu₁₀-ne',
 '16. šu!-ŋu₁₀ ab-ús šu an!-dab₅',
 '17. lú kisal-lá dub šu-tab-ba-ŋu₁₀ ki ù-ub-tag',
 '18. lú#

In [13]:
with open('Q000759b.atf', 'w', encoding = 'utf8') as w:
    w.write('\n'.join(new))

In [1]:
import os
import sys
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

In [9]:
oracc_download(['dccxt/saa31'])

http://oracc.ub.uni-muenchen.de/dccxt/saa31/json/dccxt-saa31.zip does not exist.


[]